In [25]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI()

memory = ConversationBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True
)

memory.save_context(
    {"question":"{question}"}, {"result": "{result}"}
)

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="Winston",
    chunk_size=600,
    chunk_overlap=50
    )

cache_dir = LocalFileStore("./.cache/")

loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir)

vectorstore = Chroma.from_documents(
    docs, cached_embeddings)

retriver = vectorstore.as_retriever()

map_doc_prompt = ChatPromptTemplate.from_messages([
    ("system",
    """
    Use the following portion of a long document to see if any of the text is relevant to answer the question.
    Return any relevant text verbatim.
    -----
    {context}
    """
    ),
    ("human","{question}"),
])

map_doc_chain = map_doc_prompt | llm

# context = extracted parts of a long document. 도큐멘트의 요약본
final_prompt = ChatPromptTemplate.from_messages([
    ("system",
    """
    Given the following extracted parts of a long document and a question,create a final answer.
    If you don't know the answer, just say that you don't know. Don't try to make up an answer. 
    ---
    {context}
    """
    ),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human","{question}"),
    ])

def map_docs(inputs):
    print("map_docs")
    documents = inputs['documents']
    question = inputs['question']
    return "¥n¥n".join(map_doc_chain.invoke({
        "context": doc.page_content,
        "question": question
        }).content
        for doc in documents
    )

def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]

map_chain = { "documents": retriver, "question": ["question"] } | RunnableLambda(map_docs) 

chain = RunnablePassthrough.assign(chat_history=load_memory) | {"context": map_chain,"question": ["question"]} | final_prompt | llm

def invoke_chain(question):
   result = chain.invoke({"question":question})
   memory.save_context({"question":question}, {"result": result.content})

invoke_chain("Is Aaronson guilty?")
#invoke_chain("What message did he write in the table?")
#invoke_chain("Who is Julia?")


Created a chunk of size 1830, which is longer than the specified 600
Created a chunk of size 1700, which is longer than the specified 600
Created a chunk of size 2301, which is longer than the specified 600


TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'list'>